In [13]:
from pytket_dqc.circuits import DistributedCircuit
from pytket import Circuit
from pytket.circuit import OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.distributors import Annealing, GraphPartitioning
from pytket_dqc.packing_on_output import (
    remove_packables,
    build_qubit_op_list,
    get_circuit_qubit_to_server_qubit_map,
    reinsert_cleaned_ops,
    ordered_list_to_circuit
    )
from pytket.transform import Transform
import json

Here we show some examples of 'packing' used alongside a given distributor.
If you want to quickly test it on some circuit you can use the below function `distribute_circuit_with_packing` which combines all the steps you need into one. `ebit_cost` is a quick function that will give you the ebit cost of a circuit which has SP/EPs on it.

In the (likely) case you test on a circuit that returns an error, or the output is not correct, please let me know:

1. What distributor you used.
1. The original circuit you used.
1. The network you used.
1. The JSON dict of the clean circuit if you used Annealing(). (If you specify a filename in `distribute_circuit_with_packing`, it will also dump the distributed circuit without 1 qubit diagonal/antidiagonal gates into a file so that it can be recreated).

Thanks!

In [20]:
def distribute_circuit_with_packing(circuit, network, distributor, filename = None):
    # Quick way to test out 'packing' on a distributed circuit. 
    
    clean_circuit = remove_packables(circuit) #'clean' by removing (anti)diagonals (this does not affect `circuit`)
    clean_dist_circ = DistributedCircuit(clean_circuit)
    placement = distributor.distribute(clean_dist_circ, network)
    try:
        assert clean_dist_circ.is_placement(placement)
    except:
        AssertionError('The given placement is not valid.')
    
    clean_circ_with_dist = clean_dist_circ.to_pytket_circuit(placement)
    
    if filename is not None:
        with open(filename, 'w') as f:
            json.dump(clean_circ_with_dist.to_dict(), f)
            
    ogcirc_ops = build_qubit_op_list(circuit) #ops of the original circuit
    clean_dc_ops = build_qubit_op_list(clean_circ_with_dist) #ops of the distributed 'clean' circuit
    c_qubit_to_s_qubit_map = get_circuit_qubit_to_server_qubit_map(clean_dist_circ, placement) # the map from circuit qubits to server qubits
    full_dc_ops = reinsert_cleaned_ops(ogcirc_ops, clean_dc_ops, c_qubit_to_s_qubit_map)
    full_circ_with_dist = ordered_list_to_circuit(full_dc_ops)

    return full_circ_with_dist

def ebit_cost(circuit):
    # Assumes the only CustomGates are start/end processes
    return len(circuit.commands_of_type(OpType.CustomGate)) / 2

In [21]:
circ = Circuit(5)
circ.CZ(0,1).Rx(1,0).CZ(0,1).Rx(1, 4).CZ(3,4).Rx(1,3).CZ(2,3).Rx(1,2).CZ(0,2);

In [22]:
network = NISQNetwork([[0,1], [1,2]], {0:[0, 1], 1:[2], 2:[3,4]})
distributor = Annealing()
packed_circ = distribute_circuit_with_packing(circ, network, distributor)
print(ebit_cost(packed_circ))
render_circuit_jupyter(packed_circ)

2.0


Unfortunately it is non-trivial to verify that the final output circuit is actually correct, I've just been doing it by manual comparison.

Below are some other random example circuits.

In [10]:
circ = Circuit(5)
circ.CZ(0,1).Rz(1,0).CZ(0,1).Rz(1, 4).CZ(3,4).Rz(1,3).CZ(2,3).Rz(1,2).CZ(0,2)
network = NISQNetwork([[0,1], [1,2]], {0:[0, 1], 1:[2,3], 2:[4,5]})
packed_circ = distribute_circuit_with_packing(circ, network, distributor)
render_circuit_jupyter(packed_circ)

In [11]:
circ = Circuit(5)
circ.CZ(0,1).Rz(1,0).CZ(0,1).H(4).CZ(3,4).Rx(1,3).CZ(2,3).H(2).CZ(0,2)
render_circuit_jupyter(circ)
Transform.RebaseToQuil().apply(circ)
network = NISQNetwork([[0,1], [1,2]], {0:[0, 1], 1:[2], 2:[3,4]})
packed_circ = distribute_circuit_with_packing(circ, network, distributor)
render_circuit_jupyter(packed_circ)

In [12]:
with open('examples/circuits/uccsd/raw/04_02_01.json') as f:
    circuit_s = json.load(f)
circuit = Circuit.from_dict(circuit_s)
Transform.RebaseToQuil().apply(circuit)
packed_circ = distribute_circuit_with_packing(circuit, network, distributor)
render_circuit_jupyter(packed_circ)